In [1]:
import os
import pytesseract
import fitz  # PyMuPDF
import pdfplumber
import subprocess
from pathlib import Path
from unstructured.partition.pdf import partition_pdf
from pdf2image import convert_from_path
from tempfile import TemporaryDirectory
import pandas as pd
import multiprocessing
import json


def is_scanned_pdf(pdf_path: str, sample_pages=2) -> bool:
    try:
        elements = partition_pdf(filename=pdf_path, strategy="fast", max_pages=sample_pages)
        text = "\n".join([el.text for el in elements if el.text])
        return len(text.strip()) < 50
    except Exception as e:
        print(f"[!] Unstructured failed to parse {pdf_path}: {e}")
        return True


def run_ocrmypdf(input_path: str, output_path: str):
    try:
        subprocess.run(["ocrmypdf", "--force-ocr", input_path, output_path], check=True)
        return True
    except subprocess.CalledProcessError as e:
        print(f"[!] ocrmypdf failed: {e}")
        return False


def extract_tables_from_ocr_pdf(pdf_path):
    tables = []
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for i, page in enumerate(pdf.pages):
                for table in page.extract_tables():
                    if table and any(cell for row in table for cell in row if cell and cell.strip()):
                        df = pd.DataFrame(table)
                        tables.append((i + 1, df))
    except Exception as e:
        print(f"[!] pdfplumber failed: {e}")
    return tables


def extract_text_from_native_pdf(pdf_path: str) -> str:
    elements = partition_pdf(filename=pdf_path, strategy="fast")
    return "\n".join([el.text for el in elements if el.text])


def extract_text_from_ocr_images(pdf_path):
    text = ""
    with TemporaryDirectory() as tempdir:
        images = convert_from_path(pdf_path, dpi=300, output_folder=tempdir)
        for i, image in enumerate(images):
            ocr_result = pytesseract.image_to_string(image, config="--psm 6")
            text += f"\n\n--- Page {i+1} ---\n\n" + ocr_result
    return text


def get_pdf_metadata(pdf_path: str):
    try:
        doc = fitz.open(pdf_path)
        metadata = doc.metadata or {}
        metadata["page_count"] = len(doc)
        metadata["file_path"] = str(pdf_path)
        metadata["file_name"] = Path(pdf_path).name
        return metadata
    except Exception as e:
        print(f"[!] Failed to read metadata for {pdf_path}: {e}")
        return {"file_path": pdf_path}


def hybrid_pdf_processor(args):
    pdf_path, input_root, output_root = args
    relative_path = os.path.relpath(pdf_path, input_root)
    output_txt_path = os.path.join(output_root, Path(relative_path).with_suffix(".txt"))
    output_csv_dir = os.path.join(output_root, Path(relative_path).parent)
    output_meta_path = os.path.join(output_root, Path(relative_path).with_suffix(".meta.json"))

    os.makedirs(os.path.dirname(output_txt_path), exist_ok=True)

    result_text = ""
    try:
        if is_scanned_pdf(pdf_path):
            print(f"[OCR] Detected scanned PDF: {pdf_path}")
            ocr_output_pdf = pdf_path.replace(".pdf", "_ocr.pdf")
            if run_ocrmypdf(pdf_path, ocr_output_pdf):
                tables = extract_tables_from_ocr_pdf(ocr_output_pdf)
                if tables:
                    result_text = "[Extracted Tables]\n"
                    for page_num, df in tables:
                        result_text += f"\n\n--- Page {page_num} ---\n{df.to_csv(index=False)}"
                        csv_path = os.path.join(output_csv_dir, f"{Path(pdf_path).stem}_page{page_num}.csv")
                        df.to_csv(csv_path, index=False)
                else:
                    result_text = extract_text_from_ocr_images(pdf_path)
            else:
                result_text = extract_text_from_ocr_images(pdf_path)
        else:
            print(f"[Text] Detected native text PDF: {pdf_path}")
            result_text = extract_text_from_native_pdf(pdf_path)

        with open(output_txt_path, "w", encoding="utf-8") as f:
            f.write(result_text)

        metadata = get_pdf_metadata(pdf_path)
        with open(output_meta_path, "w", encoding="utf-8") as f:
            json.dump(metadata, f, indent=2)

        print(f"[✓] Processed: {pdf_path}")
    except Exception as e:
        print(f"[✗] Failed to process {pdf_path}: {e}")


from tqdm.contrib.concurrent import process_map  # Drop-in parallel map with progress bar
from multiprocessing import Pool
from tqdm import tqdm
import multiprocessing
def batch_process_pdfs_with_multiprocessing(input_root: str, output_root: str, num_workers=4):
    pdf_paths = []
    for dirpath, _, filenames in os.walk(input_root):
        for file in filenames:
            if file.lower().endswith(".pdf"):
                pdf_paths.append(os.path.join(dirpath, file))

    args_list = [(pdf, input_root, output_root) for pdf in pdf_paths]

    with Pool(processes=num_workers) as pool:
        with tqdm(total=len(args_list), desc="Processing PDFs") as pbar:
            for _ in pool.imap_unordered(hybrid_pdf_processor, args_list):
                pbar.update()


    



In [ ]:

input_pdf_root = "/home/saranshvashistha/workspace/AIML-018-IITI-SoC/data/dataset-20250624T183251Z-1-001"  # This folder contains nested folders
output_text_root = "/home/saranshvashistha/workspace/AIML-018-IITI-SoC/text_op"

batch_process_pdfs_with_multiprocessing(input_pdf_root, output_text_root, num_workers=32)


Processing PDFs:   0%|          | 0/1302 [00:00<?, ?it/s]

[OCR] Detected scanned PDF: /home/saranshvashistha/workspace/AIML-018-IITI-SoC/data/dataset-20250624T183251Z-1-001/dataset/http_iiti.ac.in_page_former-heads-of-ee/downloads/documents/IITIPAN.pdf
[OCR] Detected scanned PDF: /home/saranshvashistha/workspace/AIML-018-IITI-SoC/data/dataset-20250624T183251Z-1-001/dataset/https_iiti.ac.in_former_directors/downloads/documents/IITIPAN.pdf
[OCR] Detected scanned PDF: /home/saranshvashistha/workspace/AIML-018-IITI-SoC/data/dataset-20250624T183251Z-1-001/dataset/https_www.iiti.ac.in_recruitments_faculty-positions/downloads/documents/IITIPAN.pdf
[Text] Detected native text PDF: /home/saranshvashistha/workspace/AIML-018-IITI-SoC/data/dataset-20250624T183251Z-1-001/dataset/http_ebsb.iiti.ac.in_/downloads/documents/ebsb.pdf


    1 page already has text! - rasterizing text and running OCR anyway


[Text] Detected native text PDF: /home/saranshvashistha/workspace/AIML-018-IITI-SoC/data/dataset-20250624T183251Z-1-001/dataset/https_iiti.ac.in_former_directors/downloads/documents/Mission_Vision_Objectives.pdf


    1 page already has text! - rasterizing text and running OCR anyway


...

Processing PDFs:  22%|██▏       | 282/1302 [1:03:35<3:02:50, 10.76s/it]

[Text] Detected native text PDF: /home/saranshvashistha/workspace/AIML-018-IITI-SoC/data/dataset-20250624T183251Z-1-001/dataset/http_www.iiti.ac.in_page_csir-ceeri/downloads/documents/Hindi_20Shabdavali.pdf
[✓] Processed: /home/saranshvashistha/workspace/AIML-018-IITI-SoC/data/dataset-20250624T183251Z-1-001/dataset/http_www.iiti.ac.in_page_csir-ceeri/downloads/documents/Hindi_20Shabdavali.pdf


Processing PDFs:  22%|██▏       | 283/1302 [1:03:52<3:34:44, 12.64s/it]

[OCR] Detected scanned PDF: /home/saranshvashistha/workspace/AIML-018-IITI-SoC/data/dataset-20250624T183251Z-1-001/dataset/https_iiti.ac.in_hindi_director/downloads/documents/IITIPAN.pdf


    1 page already has text! - rasterizing text and running OCR anyway
